In [1]:

from os.path import abspath
from pyspark.sql import SparkSession

# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

# start SparkSession with Hive support enabled
spark = SparkSession \
    .builder \
    .appName("SparkSession") \
    .master("local[1]") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

22/05/11 21:12:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:

# loading data as df to view schema
flights_df = spark.read.option('header', True) \
            .option('inferSchema', True) \
            .csv('data/flights_small.csv')

flights_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



In [3]:

# create Hive table
spark.sql("CREATE TABLE IF NOT EXISTS flights (year INT, \
                                                month INT, \
                                                day INT, \
                                                dep_time STRING, \
                                                dep_delay STRING, \
                                                arr_time STRING, \
                                                arr_delay STRING, \
                                                carrier STRING, \
                                                tailnum STRING, \
                                                flight INT, \
                                                origin STRING, \
                                                dest STRING, \
                                                air_time STRING, \
                                                distance INTEGER, \
                                                hour STRING, \
                                                minute STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH 'data/flights_small.csv' INTO TABLE flights")

22/05/11 21:12:31 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/05/11 21:12:31 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/05/11 21:12:33 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/05/11 21:12:33 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore nm@192.168.1.143
22/05/11 21:12:34 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
22/05/11 21:12:35 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
22/05/11 21:12:35 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/05/11 21:12:35 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/05/11 21:12:36 WARN ObjectStore: 

DataFrame[]

In [4]:

# Print the tables in the catalog
print(spark.catalog.listTables())

[Table(name='flights', database='default', description=None, tableType='MANAGED', isTemporary=False)]
